<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#GS-for-NB" data-toc-modified-id="GS-for-NB-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>GS for NB</a></span><ul class="toc-item"><li><span><a href="#Best-params" data-toc-modified-id="Best-params-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Best params</a></span></li></ul></li><li><span><a href="#GS-for-SVM" data-toc-modified-id="GS-for-SVM-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>GS for SVM</a></span><ul class="toc-item"><li><span><a href="#Best-params" data-toc-modified-id="Best-params-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Best params</a></span></li></ul></li><li><span><a href="#GS-for-LR" data-toc-modified-id="GS-for-LR-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>GS for LR</a></span><ul class="toc-item"><li><span><a href="#Best-params" data-toc-modified-id="Best-params-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>Best params</a></span></li></ul></li><li><span><a href="#GS-for-RF" data-toc-modified-id="GS-for-RF-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>GS for RF</a></span><ul class="toc-item"><li><span><a href="#Best-params" data-toc-modified-id="Best-params-4.1"><span class="toc-item-num">4.1&nbsp;&nbsp;</span>Best params</a></span></li></ul></li></ul></div>

In [1]:
from google.colab import drive
drive.mount('/content/drive')
import os
os.chdir("/content/drive/MyDrive/Text-Classification/code")
!pip install pyLDAvis
!pip install gensim
!pip install pandas==1.3.0
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('words')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Package words is already up-to-date!


True

In [2]:
%%javascript
IPython.notebook.kernel.execute('nb_name = "' + IPython.notebook.notebook_name + '"')

<IPython.core.display.Javascript object>

In [3]:
nb_name = "svm_nb_lr_RF_baseline_v2_voc_only"

In [4]:
import numpy as np 
import pandas as pd
import random

from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import GridSearchCV
import xgboost as xgb
from sklearn.pipeline import Pipeline

from classification_utils import *
from clustering_utils import *
from eda_utils import *
from nn_utils_keras import *
from feature_engineering_utils import *
from data_utils import *
setup_seed_ml(2021)


/usr/local/lib/python3.7/dist-packages/past/types/oldstr.py:5: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3,and in 3.9 it will stop working
  from collections import Iterable


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Package words is already up-to-date!


In [5]:
train, test = load_data(only_stem_voc=True)
# train, upsampling_info = upsampling_train(train)

# train_text, train_label = train_augmentation(train, select_comb=[['text'], ['reply', 'reference_one']])
train_text, train_label = train_augmentation(train, select_comb=[['text']])
test_text, test_label = test['text'], test['label']

# test_text = test_text.apply(lambda x: normal_string(x))
# train_text = train_text.apply(lambda x: normal_string(x))


may use cols: 
 ['global_index', 'doc_path', 'label', 'reply', 'reference_one', 'reference_two', 'tag_reply', 'tag_reference_one', 'tag_reference_two', 'Subject', 'From', 'Lines', 'Organization', 'contained_emails', 'long_string', 'text', 'error_message']
combination 1 train:  ['text']


In [6]:
####################################
### label mapper
####################################
labels = sorted(train_label.unique())
label_mapper = dict(zip(labels, range(len(labels))))
train_label = train_label.map(label_mapper)
test_label = test_label.map(label_mapper)
y_train = train_label.values
y_test = test_label.values

print(train_text.shape)
print(test_text.shape)
print(train_label.shape)
print(test_label.shape)

(11083,)
(7761,)
(11083,)
(7761,)


# GS for NB

In [ ]:
metric = "f1_macro"

text_clf = Pipeline([('tfidf', TfidfVectorizer()), ('clf', MultinomialNB())])
parameters = {'tfidf__min_df': [1, 3], 'tfidf__stop_words': [ 'english'], 'tfidf__use_idf': [True], 'tfidf__binary': [True],
              'clf__alpha': [0.2,0.4,0.6, 0.8, 1]}
gs_clf = GridSearchCV(text_clf, scoring=metric, param_grid=parameters, cv=4)
gs_clf = gs_clf.fit(train_text, y_train)

for param_name in gs_clf.best_params_:
    print("{0}:\t{1}".format(param_name, gs_clf.best_params_[param_name]))

print("best f1 score: {:.3f}".format(gs_clf.best_score_))
cv_results = pd.DataFrame(gs_clf.cv_results_)
cv_results.to_excel(f"NB_cv_result_{nb_name}.xlsx")
set(y_test)

clf__alpha:	0.2
tfidf__binary:	True
tfidf__min_df:	3
tfidf__stop_words:	english
tfidf__use_idf:	True
best f1 score: 0.758


{0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19}

## Best params

In [ ]:
X_train, X_test, word_to_idx, tfidf_vect = tfidf_vectorizer(train_text, test_text, stop_words=True, binary=True, min_df=3)
# X_train, X_test, word_to_idx, tfidf_vect = count_vectorizer(train_text, test_text, binary=True, min_df=1)
# tfidf_vectorizer(train_text, test_text, min_df=2, max_df=100)
# X_train, transform_mapper = dimension_reduction(X_train, out_dim=500) # not allow negative 
# X_test = transform_mapper.transform(X_test)

print('X_train.shape', X_train.shape)
print('X_test.shape', X_test.shape)


clf = MultinomialNB(alpha=0.2)
clf.fit(X_train, y_train)
pred = clf.predict(X_test)


classification_report = evaluation_report(y_test, pred, labels=labels)
cv = clf_cv(clf, X_train, y_train, kfold=4, cv_metrics=["precision_macro", "accuracy", "f1_macro", "f1_micro"])
pred_proba = clf.predict_proba(X_test)
roc_auc(y_test, pred_proba)
cv

num of words: 6909
X_train.shape (11083, 6909)
X_test.shape (7761, 6909)
classification_report:
f1:  {'micro avg': 0.6852209766782631, 'macro avg': 0.681659458040722} 

                          precision  recall  f1-score  support
alt.atheism                   0.684   0.630     0.656    319.0
comp.graphics                 0.653   0.635     0.644    389.0
comp.os.ms-windows.misc       0.606   0.538     0.570    394.0
comp.sys.ibm.pc.hardware      0.540   0.673     0.599    392.0
comp.sys.mac.hardware         0.676   0.613     0.643    385.0
comp.windows.x                0.650   0.691     0.670    395.0
misc.forsale                  0.795   0.746     0.770    390.0
rec.autos                     0.785   0.795     0.790    395.0
rec.motorcycles               0.837   0.864     0.850    398.0
rec.sport.baseball            0.425   0.927     0.583    397.0
rec.sport.hockey              1.000   0.336     0.503    827.0
sci.crypt                     0.823   0.881     0.851    396.0
sci.electron

,fit_time,score_time,test_precision_macro,train_precision_macro,test_accuracy,train_accuracy,test_f1_macro,train_f1_macro,test_f1_micro,train_f1_micro
0,0.018465,0.009516,0.766054,0.951552,0.748466,0.947185,0.728188,0.942150,0.748466,0.947185
1,0.018756,0.009372,0.823949,0.944743,0.803681,0.939726,0.792816,0.932093,0.803681,0.939726
2,0.018839,0.009216,0.813071,0.947489,0.795742,0.942733,0.773618,0.937633,0.795742,0.942733
3,0.019058,0.009134,0.780224,0.949255,0.770758,0.944785,0.743315,0.940757,0.770758,0.944785


# GS for SVM

In [ ]:
metric = "f1_macro"

text_clf = Pipeline([('tfidf', TfidfVectorizer()), ('clf', LinearSVC())])
parameters = {'tfidf__min_df': [1, 3], 'tfidf__stop_words': ['english'], 'tfidf__use_idf': [True], 'tfidf__binary': [True],
             'clf__penalty':['l2'], 'clf__C':[1,2,3,4,5]}
gs_clf = GridSearchCV(text_clf, scoring=metric, param_grid=parameters, cv=4)
gs_clf = gs_clf.fit(train_text, y_train)

for param_name in gs_clf.best_params_:
    print("{0}:\t{1}".format(param_name, gs_clf.best_params_[param_name]))

print("best f1 score: {:.3f}".format(gs_clf.best_score_))
cv_results = pd.DataFrame(gs_clf.cv_results_)
cv_results.to_excel(f"SVC_cv_result_{nb_name}.xlsx")

clf__C:	1
clf__penalty:	l2
tfidf__binary:	True
tfidf__min_df:	1
tfidf__stop_words:	english
tfidf__use_idf:	True
best f1 score: 0.804


## Best params

In [ ]:
X_train, X_test, word_to_idx, tfidf_vect = tfidf_vectorizer(train_text, test_text, stop_words=False, binary=True, min_df=3)
# X_train, X_test, word_to_idx, tfidf_vect = count_vectorizer(train_text, test_text, binary=True, min_df=1)
# X_train, transform_mapper = dimension_reduction(X_train, out_dim=1000)
# X_test = transform_mapper.transform(X_test)

print('X_train.shape', X_train.shape)
print('X_test.shape', X_test.shape)


clf = LinearSVC(penalty="l2", multi_class='ovr', C=3, dual=True)
clf.fit(X_train, y_train)
pred = clf.predict(X_test)


classification_report = evaluation_report(y_test, pred, labels=labels)
cv = clf_cv(clf, X_train, y_train, kfold=4, cv_metrics=["precision_macro", "accuracy", "f1_macro", "f1_micro"])
cv

num of words: 7126
X_train.shape (11083, 7126)
X_test.shape (7761, 7126)
classification_report:
f1:  {'micro avg': 0.7237469398273418, 'macro avg': 0.7100411127099727} 

                          precision  recall  f1-score  support
alt.atheism                   0.671   0.639     0.655    319.0
comp.graphics                 0.614   0.604     0.609    389.0
comp.os.ms-windows.misc       0.572   0.574     0.573    394.0
comp.sys.ibm.pc.hardware      0.553   0.599     0.575    392.0
comp.sys.mac.hardware         0.633   0.655     0.644    385.0
comp.windows.x                0.698   0.638     0.667    395.0
misc.forsale                  0.757   0.800     0.778    390.0
rec.autos                     0.781   0.785     0.783    395.0
rec.motorcycles               0.909   0.882     0.895    398.0
rec.sport.baseball            0.705   0.904     0.792    397.0
rec.sport.hockey              0.985   0.785     0.873    827.0
sci.crypt                     0.872   0.843     0.858    396.0
sci.electro

,fit_time,score_time,test_precision_macro,train_precision_macro,test_accuracy,train_accuracy,test_f1_macro,train_f1_macro,test_f1_micro,train_f1_micro
0,1.096067,0.012324,0.747442,0.999547,0.743414,0.999519,0.742645,0.999545,0.743414,0.999519
1,1.142490,0.013961,0.825866,0.998869,0.822808,0.998797,0.823001,0.998868,0.822808,0.998797
2,1.209999,0.011588,0.819429,0.999436,0.814868,0.999398,0.811265,0.999434,0.814868,0.999398
3,1.197793,0.011181,0.778622,0.999209,0.779061,0.999158,0.776164,0.999208,0.779061,0.999158


In [ ]:
X_train, X_test, word_to_idx, tfidf_vect = tfidf_vectorizer(train_text, test_text, dense=False, stop_words=True, binary=True, min_df=1)
# X_train, X_test, word_to_idx, tfidf_vect = count_vectorizer(train_text, test_text, binary=True, min_df=1)
# X_train, transform_mapper = dimension_reduction(X_train, out_dim=1000)
# X_test = transform_mapper.transform(X_test)

print('X_train.shape', X_train.shape)
print('X_test.shape', X_test.shape)


clf = LinearSVC(penalty="l2", multi_class='ovr', C=3.0, dual=True)
clf.fit(X_train, y_train)
pred = clf.predict(X_test)


classification_report = evaluation_report(y_test, pred, labels=labels)
# cv = clf_cv(clf, X_train, y_train, kfold=4, cv_metrics=["precision_macro", "accuracy", "f1_macro", "f1_micro"])
# cv

num of words: 10587
X_train.shape (11083, 10587)
X_test.shape (7761, 10587)
classification_report:
f1:  {'micro avg': 0.7209122535755702, 'macro avg': 0.7068757549337542} 

                          precision  recall  f1-score  support
alt.atheism                   0.679   0.636     0.657    319.0
comp.graphics                 0.613   0.614     0.614    389.0
comp.os.ms-windows.misc       0.600   0.556     0.577    394.0
comp.sys.ibm.pc.hardware      0.548   0.612     0.578    392.0
comp.sys.mac.hardware         0.633   0.660     0.646    385.0
comp.windows.x                0.665   0.613     0.638    395.0
misc.forsale                  0.715   0.785     0.748    390.0
rec.autos                     0.779   0.757     0.768    395.0
rec.motorcycles               0.916   0.882     0.899    398.0
rec.sport.baseball            0.701   0.887     0.783    397.0
rec.sport.hockey              0.982   0.799     0.881    827.0
sci.crypt                     0.878   0.833     0.855    396.0
sci.elec

# GS for LR

In [ ]:
metric = "f1_macro"

text_clf = Pipeline([('tfidf', TfidfVectorizer()), ('clf', LogisticRegression( multi_class='ovr', max_iter=1000))])
parameters = {'tfidf__min_df': [1, 3, 5], 'tfidf__stop_words': ['english'], 'tfidf__use_idf': [True], 'tfidf__binary': [True],
             'clf__penalty':['l2'], 'clf__C':[1,2,3], 'clf__solver': ['liblinear', 'newton-cg', 'sag', 'saga' , 'lbfgs']}
gs_clf = GridSearchCV(text_clf, scoring=metric, param_grid=parameters, cv=2)
gs_clf = gs_clf.fit(train_text, y_train)

for param_name in gs_clf.best_params_:
    print("{0}:\t{1}".format(param_name, gs_clf.best_params_[param_name]))

print("best f1 score: {:.3f}".format(gs_clf.best_score_))
cv_results = pd.DataFrame(gs_clf.cv_results_)
cv_results.to_excel(f"LR_1_cv_result_{nb_name}.xlsx")

Streaming output truncated to the last 5000 lines.
/usr/local/lib/python3.7/dist-packages/scipy/optimize/lbfgsb.py:339: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.
  task_str = task.tostring()
/usr/local/lib/python3.7/dist-packages/scipy/optimize/lbfgsb.py:339: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.
  task_str = task.tostring()
/usr/local/lib/python3.7/dist-packages/scipy/optimize/lbfgsb.py:339: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.
  task_str = task.tostring()
/usr/local/lib/python3.7/dist-packages/scipy/optimize/lbfgsb.py:339: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.
  task_str = task.tostring()
/usr/local/lib/python3.7/dist-packages/scipy/optimize/lbfgsb.py:339: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.
  task_str = task.tostring()
/usr/local/lib/python3.7/dist-packages/scipy/optimize/lbfgsb.py:339: DeprecationWarning: tostring() is deprecated.

clf__C:	3
clf__penalty:	l2
clf__solver:	saga
tfidf__binary:	True
tfidf__min_df:	3
tfidf__stop_words:	english
tfidf__use_idf:	True
best f1 score: 0.752


In [ ]:
metric = "f1_macro"

text_clf = Pipeline([('tfidf', TfidfVectorizer()), ('clf', LogisticRegression( multi_class='ovr'))])
parameters = {'tfidf__min_df': [1, 3, 5], 'tfidf__stop_words': [None, 'english'], 'tfidf__use_idf': [True, False], 'tfidf__binary': [True, False],
             'clf__penalty':['elasticnet'], 'clf__C':[1,2,3], 'clf__l1_ratio':[0.2, 0.5,0.8],'clf__solver': ['saga']}
gs_clf = GridSearchCV(text_clf, scoring=metric, param_grid=parameters, cv=2)
gs_clf = gs_clf.fit(train_text, y_train)

for param_name in gs_clf.best_params_:
    print("{0}:\t{1}".format(param_name, gs_clf.best_params_[param_name]))

print("best f1 score: {:.3f}".format(gs_clf.best_score_))
cv_results = pd.DataFrame(gs_clf.cv_results_)
cv_results.to_excel(f"LR_2_cv_result_{nb_name}.xlsx")

Streaming output truncated to the last 5000 lines.
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_sag.py:354: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  ConvergenceWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_sag.py:354: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  ConvergenceWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_sag.py:354: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  ConvergenceWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_sag.py:354: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  ConvergenceWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_sag.py:354: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  ConvergenceWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/linear_

clf__C:	3
clf__l1_ratio:	0.2
clf__penalty:	elasticnet
clf__solver:	saga
tfidf__binary:	True
tfidf__min_df:	1
tfidf__stop_words:	english
tfidf__use_idf:	True
best f1 score: 0.748


/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_sag.py:354: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  ConvergenceWarning,


## Best params

In [ ]:
X_train, X_test, word_to_idx, tfidf_vect = tfidf_vectorizer(train_text, test_text, binary=True, min_df=1)
# X_train, transform_mapper = dimension_reduction(X_train, out_dim=1000)
# X_test = transform_mapper.transform(X_test)

print('X_train.shape', X_train.shape)
print('X_test.shape', X_test.shape)


clf = LogisticRegression(penalty="l2", multi_class='ovr', max_iter=1000)
clf.fit(X_train, y_train)
pred = clf.predict(X_test)


classification_report = evaluation_report(y_test, pred, labels=labels)
cv = clf_cv(clf, X_train, y_train, kfold=4, cv_metrics=["precision_macro", "accuracy", "f1_macro", "f1_micro"])
pred_proba = clf.predict_proba(X_test)
roc_auc(y_test, pred_proba)
cv

num of words: 10809
X_train.shape (11083, 10809)
X_test.shape (7761, 10809)


/usr/local/lib/python3.7/dist-packages/scipy/optimize/lbfgsb.py:339: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.
  task_str = task.tostring()
/usr/local/lib/python3.7/dist-packages/scipy/optimize/lbfgsb.py:339: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.
  task_str = task.tostring()
/usr/local/lib/python3.7/dist-packages/scipy/optimize/lbfgsb.py:339: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.
  task_str = task.tostring()
/usr/local/lib/python3.7/dist-packages/scipy/optimize/lbfgsb.py:339: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.
  task_str = task.tostring()
/usr/local/lib/python3.7/dist-packages/scipy/optimize/lbfgsb.py:339: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.
  task_str = task.tostring()
/usr/local/lib/python3.7/dist-packages/scipy/optimize/lbfgsb.py:339: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.
  task_str = task.tostring(

classification_report:
f1:  {'micro avg': 0.69900785981188, 'macro avg': 0.6935496092677548} 

                          precision  recall  f1-score  support
alt.atheism                   0.671   0.596     0.631    319.0
comp.graphics                 0.640   0.653     0.646    389.0
comp.os.ms-windows.misc       0.686   0.678     0.682    394.0
comp.sys.ibm.pc.hardware      0.664   0.645     0.655    392.0
comp.sys.mac.hardware         0.704   0.660     0.681    385.0
comp.windows.x                0.728   0.656     0.690    395.0
misc.forsale                  0.746   0.856     0.797    390.0
rec.autos                     0.799   0.797     0.798    395.0
rec.motorcycles               0.936   0.877     0.905    398.0
rec.sport.baseball            0.439   0.929     0.597    397.0
rec.sport.hockey              0.991   0.404     0.574    827.0
sci.crypt                     0.896   0.828     0.861    396.0
sci.electronics               0.648   0.631     0.639    393.0
sci.med                

Streaming output truncated to the last 5000 lines.
/usr/local/lib/python3.7/dist-packages/scipy/optimize/lbfgsb.py:339: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.
  task_str = task.tostring()
/usr/local/lib/python3.7/dist-packages/scipy/optimize/lbfgsb.py:360: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.
  task_str = task.tostring().strip(b'\x00').strip()
/usr/local/lib/python3.7/dist-packages/scipy/optimize/lbfgsb.py:339: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.
  task_str = task.tostring()
/usr/local/lib/python3.7/dist-packages/scipy/optimize/lbfgsb.py:339: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.
  task_str = task.tostring()
/usr/local/lib/python3.7/dist-packages/scipy/optimize/lbfgsb.py:339: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.
  task_str = task.tostring()
/usr/local/lib/python3.7/dist-packages/scipy/optimize/lbfgsb.py:339: DeprecationWarning: to

cv average f1 macro:  0.7602296562850255
cv average f1 micro:  0.778307659135945
roc_auc:  0.9695600577713025


,fit_time,score_time,test_precision_macro,train_precision_macro,test_accuracy,train_accuracy,test_f1_macro,train_f1_macro,test_f1_micro,train_f1_micro
0,10.256285,0.029588,0.777743,0.938752,0.754962,0.933470,0.738464,0.929386,0.754962,0.933470
1,11.081234,0.018254,0.816440,0.930413,0.798268,0.923845,0.788176,0.917646,0.798268,0.923845
2,9.466762,0.036444,0.811133,0.928806,0.797907,0.922642,0.777711,0.918115,0.797907,0.922642
3,9.799820,0.037960,0.772384,0.930065,0.762094,0.923734,0.736568,0.918651,0.762094,0.923734


# GS for RF

In [10]:
from xgboost import XGBClassifier


NameError: ignored

In [ ]:
X_train, X_test, word_to_idx, tfidf_vect = tfidf_vectorizer(train_text, test_text, dense=False, stop_words=True, binary=True, min_df=1)
# X_train, X_test, word_to_idx, tfidf_vect = count_vectorizer(train_text, test_text, binary=True, min_df=1)
# X_train, transform_mapper = dimension_reduction(X_train, out_dim=1000)
# X_test = transform_mapper.transform(X_test)

print('X_train.shape', X_train.shape)
print('X_test.shape', X_test.shape)

metric = "f1_macro"

text_clf = Pipeline([('tfidf', TfidfVectorizer()), 
                     ('clf', XGBClassifier(objective='multi:softmax', n_estimators=500, num_class=20, learning_rate=0.075, \
                                           colsample_bytree=0.7, subsample=0.8, eval_metric='merror'))])

parameters = {
    'tfidf__min_df': [1, 3], 'tfidf__stop_words': ['english'], 'tfidf__use_idf': [True], 'tfidf__binary': [True],
    # 'svd__n_components': (250, 500, 750),
    'clf__n_estimators': (250, 500, 750),
    'clf__max_depth': (4, 6, 8),
    'clf__min_child_weight': (1, 5, 10),
    'clf__alpha': (0.00001, 0.000001),
    'clf__penalty': ('l2', 'elasticnet'),
    'clf__max_iter': (10, 50, 80),
}

gs_clf = GridSearchCV(text_clf, scoring=metric, param_grid=parameters, cv=2)
gs_clf = gs_clf.fit(train_text, y_train)

for param_name in gs_clf.best_params_:
    print("{0}:\t{1}".format(param_name, gs_clf.best_params_[param_name]))

print("best f1 score: {:.3f}".format(gs_clf.best_score_))
cv_results = pd.DataFrame(gs_clf.cv_results_)
cv_results.to_excel(f"RF_1_cv_result_{nb_name}.xlsx")

num of words: 10587
X_train.shape (11083, 10587)
X_test.shape (7761, 10587)


## Best params

In [ ]:
from xgboost import XGBClassifier

In [ ]:
X_train, X_test, word_to_idx, tfidf_vect = tfidf_vectorizer(
    train_text, test_text, binary=True, min_df=3)
# X_train, X_test, word_to_idx, tfidf_vect = count_vectorizer(train_text, test_text, binary=True, min_df=1)
# X_train, transform_mapper = dimension_reduction(X_train, out_dim=500) # not allow negative
# X_test = transform_mapper.transform(X_test)

print('X_train.shape', X_train.shape)
print('X_test.shape', X_test.shape)

# param = {'max_depth': 4, 'objective': 'multi:softmax', 'clf__min_child_weight': 1,
#          'booster':'gblinear', # linear model
#          'alpha': 0.0001,  # alpha is the L1 regularizer
#          'lambda': 1} # lambda is the L2 regularizer
# param['num_class'] = len(labels)
# param['eta'] = 0.1 # # scale weight of positive examples
# param['eval_metric'] = ['auc', 'ams@0']


clf = XGBClassifier(objective='multi:softmax', max_depth=4, min_child_weight=1, n_estimators=500, num_class=20, learning_rate=0.075,
                    colsample_bytree=0.7, subsample=0.8, eval_metric='merror')
# clf =  xgb.XGBClassifier(param)

clf.fit(X_train, y_train)
pred = clf.predict(X_test)

In [ ]:
classification_report = evaluation_report(y_test, pred, labels=labels)
cv = clf_cv(clf, X_train, y_train, kfold=4, cv_metrics=["precision_macro", "accuracy", "f1_macro", "f1_micro"])
pred_proba = clf.predict_proba(X_test)
roc_auc(y_test, pred_proba)
cv

In [ ]:
# clf.save_model("xgb.bin")
# clf.load_model("xgb.bin")
xgb.plot_importance(clf)
xgb.to_graphviz(clf, num_trees=2)